# 1.How to run the repository in Linux？

linux: Ubuntu 5.4.0-6ubuntu1~16.04.9 （cat /proc/version）<br>
python: Python 2.7.15 |Anaconda, Inc.| (default, May  1 2018, 23:32:55) <br>
conda：conda 4.5.9

In [ ]:
git clone https://github.com/luispedro/diskhash.git

In [ ]:
git clone https://github.com/luispedro/fasta_reader.git

In [ ]:
git clone https://github.com/BigDataBiology/kmer.find.git

## 1.1kmer.find install

In [ ]:
cd kmer.find

In [ ]:
conda env create -f environmnent.yml 

In [ ]:
source activate ngless

In [ ]:
make

In [ ]:
 ./build.kmer-ix.sh test.faa

pkg_resources.DistributionNotFound: The 'diskhash' distribution was not found and is required by fasta-reader.<br>
Then I tried to install the the two python packages.

In [ ]:
#build.kmer-ix.sh
#!/usr/bin/env bash

set -e   #This tells bash that it should exit the script if any statement returns a non-true return value. 
export PATH=$PWD/bin/:$PATH

input_faa="$1"  

if ! test -f ${input_faa}.dhi; then     #File exists and is a regular file
	echo "FASTA file not indexed. Indexing..."
	index_fasta ${input_faa} 
    #file: fasta_reader/index_fasta.py,
    #tb.insert(header, position_start, (seq_len << 1) + multiline)
    #import argparse    #https://docs.python.org/3/library/argparse.html
    #from diskhash import StructHash
    #StructHash: /root/miniconda3/envs/ngless/lib/python3.6/site-packages/diskhash-0.4.0-py3.6-linux-x86_64.egg/diskhash
fi

mkdir kmer.index       #The term k-mer typically refers to all the possible substrings of length k that are contained in a string.
echo "starting..."
EncodeKmers -i ${input_faa} -o kmer.index/${input_faa}.kmer.ix -t8  # what???
echo "Encoding kmers done"


python extract-names32.py ${input_faa} kmer.index/${input_faa}.names.32
cd kmer.index
du -sh *

for k in *.kmer.ix.*; do
    sort32pairs $k ${k}.sorted
    rm ${k}
    echo "Sorted ${k}"
done

cat ${input_faa}.kmer.ix.0.sorted \
    ${input_faa}.kmer.ix.1.sorted \
    ${input_faa}.kmer.ix.2.sorted \
    ${input_faa}.kmer.ix.3.sorted \
    ${input_faa}.kmer.ix.4.sorted \
    ${input_faa}.kmer.ix.5.sorted \
    ${input_faa}.kmer.ix.6.sorted \
    ${input_faa}.kmer.ix.7.sorted \
    ${input_faa}.kmer.ix.8.sorted \
    ${input_faa}.kmer.ix.9.sorted \
    ${input_faa}.kmer.ix.10.sorted \
    ${input_faa}.kmer.ix.11.sorted \
    ${input_faa}.kmer.ix.12.sorted \
    ${input_faa}.kmer.ix.13.sorted \
    ${input_faa}.kmer.ix.14.sorted \
    ${input_faa}.kmer.ix.15.sorted \
        > ${input_faa}.kmer.ix.sorted

BuildIndex2 -i ${input_faa}.kmer.ix.sorted -o ${input_faa}.kmer.ix1 -p ${input_faa}.kmer.ix2 -v -t8

## 1.2 diskhash and fasta_reader install

In [ ]:
cd ../diskhash

In [ ]:
python setup.py build

In [ ]:
python setup.py install

Installing library in
/mnt/data2/jialonghao/sequence-match/diskhash/.stack-work/install/x86_64-linux/lts-8.15/8.0.2/lib/x86_64-linux-ghc-8.0.2/diskhash-0.0.4.0-BMEnkiPN1SG83m20ylUIH<br>
Registering diskhash-0.0.4.0...

In [ ]:
#diskhash


In [ ]:
cd ../fasta_reader

In [ ]:
python setup.py build

In [ ]:
python setup.py install

Installed /root/miniconda3/envs/ngless/lib/python3.6/site-packages/diskhash-0.4.0-py3.6-linux-x86_64.egg <br>
Finished processing dependencies for fasta-reader==0.1.2

## 1.3 run for the test

In [ ]:
cd ../kmer.find

In [ ]:
 ./build.kmer-ix.sh test.faa

FASTA file not indexed. Indexing...<br>
Creating diskhash index for test.faa.<br>
...<br>
CmdArgs {ifileArg = "test.faa.kmer.ix.sorted", ofileArg1 = "test.faa.kmer.ix1", ofileArg2 = "test.faa.kmer.ix2", verboseArg = True, nJobsArg = 8}

# 2. Make some improvement

reference:[Score,Bit-score,P-value,E-value](https://docplayer.net/22262214-Score-bit-score-p-value-e-value.html)

In [ ]:
export PATH=$PWD/bin/:$PATH
python query.py test.faa query.faa

The result:<br>
Loaded headers<br>
Q1	GMGC10.016_126_784.UNKNOWN	99<br>
Q1	GMGC10.046_029_341.UNKNOWN	17<br>
Q1	GMGC10.045_665_422.UNKNOWN	14<br>
Q1	GMGC10.029_051_291.UNKNOWN	13<br>
Q1	GMGC10.032_042_208.UNKNOWN	12<br>
Q1	GMGC10.031_644_364.UNKNOWN	10<br>

## 2.1 Caculate e-value and bitscore

In [ ]:
from os import path
from itertools import chain
from fasta_reader import IndexedFastaReader
import skbio.alignment
import skbio.sequence
import skbio.io
from sys import argv
import subprocess
import math

index_fname = argv[1]
query_fname = argv[2]

index_base, index_fname  = path.split(path.abspath(index_fname))

class GetNames(object):
    def __init__(self, fname):
        self.headers = open(fname, 'rb')
    def get(self, ix):
        self.headers.seek(32 * ix)
        return self.headers.readline().decode('ascii').strip()
        
Qs = {}
for fa in skbio.io.read(query_fname, format='fasta'):
    Qs[fa.metadata['id']] = str(fa)

    
headers = GetNames(f'{index_base}/kmer.index/{index_fname}.names.32')
print("Loaded headers") 
    
index = IndexedFastaReader(path.join(index_base, index_fname))
    
data = subprocess.Popen(['Query',
                '-i', query_fname,
                '-o', '/dev/stdout',
                '-1', f'{index_base}/kmer.index/{index_fname}.kmer.ix1',
                '-2', f'{index_base}/kmer.index/{index_fname}.kmer.ix2'],
        stdout=subprocess.PIPE)

matches = []
alpha = 0.5772 #λ is the Gumble distribution constant
K = 0.13    # K is a constant associated with the scoring matrix used.
query_sequence = 450     #the query sequence length(n)
database_sequence = 83708   #the sum of the lengths of the sequences in the database(m)
for line in chain(data.stdout, [b'END']):
    if line.startswith(b'CmdArgs'): continue
    line = line.decode('ascii')
    if line[0] == '>' or line == 'END':
        if len(matches):
            matches.sort(key=lambda m: m[1]['optimal_alignment_score'], reverse=True)
            for fah, m in matches:
                bit_score = (alpha * m.optimal_alignment_score - math.log(K)) / math.log(2)	
                p_value = math.pow(2, (-bit_score))
                E_value = database_sequence * query_sequence * p_value
                #E_value = K * database_sequence * query_sequence * exp(-alpha *  m.optimal_alignment_score)
                print(f'{active}\t{fah}\t{m.optimal_alignment_score}\t{bit_score}\t{E_value}')
            matches = []
        if line == "END":
            break
        active = line[1:].strip()
        fa = Qs[active]
        sw = skbio.alignment.StripedSmithWaterman(fa)
    else:
        name = headers.get(int(line.strip()))
        seq = index.get(name).decode('ascii')
        matches.append((name, sw(seq)))

The result:
Loaded headers
Q1	GMGC10.016_126_784.UNKNOWN	99	85.3830506541435	7.466552295325474e-19
Q1	GMGC10.046_029_341.UNKNOWN	17	17.099717290852496	268.19560551216534
Q1	GMGC10.045_665_422.UNKNOWN	14	14.601546558049167	1515.2210056155707
Q1	GMGC10.029_051_291.UNKNOWN	13	13.76882298044806	2698.6760654353084
Q1	GMGC10.032_042_208.UNKNOWN	12	12.93609940284695	4806.462211890144
Q1	GMGC10.031_644_364.UNKNOWN	10	11.27065224764473	15246.672867330346